In [67]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import yfinance as yf
import math
from statistics import median
import scipy.stats as st
cdf = st.norm.cdf
import option_util_V2 as ou
sqrt = math.sqrt
from importlib import reload
import re
import itertools
from multiprocessing import Pool
import multiprocessing
import time
from tqdm.auto import tqdm, trange
from IPython.display import clear_output
clear_output(wait=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


#from pandas_datareader import data as pdr

packages = {'pd': pd, 'np': np, 'cdf': cdf, 're': re, 'sqrt': sqrt,\
           'itertools': itertools, 'ou': ou}

In [32]:
# Grab Historical Price Data 
stock = yf.Ticker("W")
hist_data = stock.history(period="240mo",
                    interval="1d")

hist_data.tail()

latest_date = hist_data.index.strftime('%Y-%m-%d')[-1]
latest_date

'2020-06-05'

In [33]:

# First get the list of expirations

stock.options

('2020-06-11',
 '2020-06-18',
 '2020-06-25',
 '2020-07-01',
 '2020-07-09',
 '2020-07-16',
 '2020-08-20',
 '2020-11-19',
 '2021-01-14',
 '2022-01-20')

In [34]:
# querying calls

exp_date = stock.options[4]  # taking a month out 
opt = stock.option_chain(exp_date)

# calls -- look at the middle (insetad of head/tail)
calls = opt.calls[ (opt.calls['volume'] >= 0) & (opt.calls['openInterest'] >= 0) ]  # vol 15  op 50
inmoneycall = len(calls[ calls['inTheMoney'] == True])
print('exp date: ', exp_date)
calls.iloc[inmoneycall-2:inmoneycall+5]
calls

exp date:  2020-07-09


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
1,W200710C00160000,2020-06-04 14:05:53,160.0,32.21,20.6,24.5,0.000000,0.000000,1.0,5,0.827761,True,REGULAR,USD
2,W200710C00165000,2020-06-01 15:44:54,165.0,19.20,17.2,21.5,0.000000,0.000000,6.0,0,0.799807,True,REGULAR,USD
3,W200710C00170000,2020-06-05 15:50:44,170.0,15.96,15.5,19.0,-7.839999,-32.941174,1.0,2,0.815126,True,REGULAR,USD
4,W200710C00172500,2020-06-05 13:47:16,172.5,21.70,14.0,17.5,1.450001,7.160498,1.0,1,0.796511,False,REGULAR,USD
5,W200710C00185000,2020-06-05 14:12:42,185.0,11.17,7.7,11.5,-3.930000,-26.026491,4.0,29,0.726931,False,REGULAR,USD
6,W200710C00190000,2020-06-05 19:26:23,190.0,8.40,6.3,9.7,-8.370001,-49.910557,12.0,13,0.721072,False,REGULAR,USD
7,W200710C00200000,2020-06-05 19:57:10,200.0,5.00,4.8,7.5,-2.200000,-30.555553,1.0,9,0.751711,False,REGULAR,USD


In [35]:
# PUTS

# Puts with min volume & interest
puts = opt.puts[ (opt.puts['volume'] >= 0) & (opt.puts['openInterest'] >= 0) ]   # vol 15 op 50

# 
inmoneyput = len(puts[puts['inTheMoney']==False])
puts.iloc[inmoneyput-5:inmoneyput+2]


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
2,W200710P00155000,2020-06-05 19:57:20,155.0,9.6,7.9,11.7,3.1,47.692314,1,7,0.834718,False,REGULAR,USD
3,W200710P00175000,2020-05-29 16:17:20,175.0,21.5,16.2,19.5,0.0,0.000000,8,6,0.744631,True,REGULAR,USD



## It Starts


In [36]:
# Single Output Inputs

exp_date = exp_date
#exp_date = stock.options[4]  # grabbing the option end date 1 month out 

# CALL AND PUT INPUTS -- FLOATS ONLY -- input None if no option -> e.g. sell_call = None
buy_call_strike =  180 # 315.0
buy_put_strike = 170 # 315.0
sell_call_strike = 190 #317.5 # 325.0
sell_put_strike = 160 # 305.0 # = None

opt_strikes = [buy_call_strike, buy_put_strike, sell_call_strike, sell_put_strike]

# CURRENT PRICE

current_price = hist_data['Close'].tail(1)[0]

# Price percent range to look at
perc_range, increments = .40, .02

print('current price: '+str(current_price))
print('exp date: '+str(exp_date))


# Live Market Data for options

live = input('What Price Do You Want? 0 Worst Bid/Ask; 1 LastTradedPrice; 2 Mid Bid/Ask:  ')

current price: 170.76
exp date: 2020-07-09
What Price Do You Want? 0 Worst Bid/Ask; 1 LastTradedPrice; 2 Mid Bid/Ask:  0


## Standard Deviation of the Stock Returns & Price

In [37]:
# Getting STD (from another file)

stock_data, final_ret_std, mthly_ret_std, price_std_pct, price_std = ou.get_std(hist_data, 20, exp_date) # days for STD

# Stock_data, std = io.get_std(hist_data, 15, 20, sqrt) 
# Takes: DF  \  STD over X days \ sqrt function
# Returns: DF \ STD of returns (over Y period) over last X days

print('final STD: ', round(final_ret_std,4))   # this is the avg daily chg
print('mythly_ret_std: ', round(mthly_ret_std,4))
print('price_std_pct: ', round(price_std_pct,4))
print('price_std: ', round(price_std,4))

final STD:  0.0517
mythly_ret_std:  0.2531
price_std_pct:  0.1854
price_std:  28.5282


## Grab Price Data and Calculate Outcomes based on Probabilities

In [38]:
# Get Option Price Data

opt_prices = ou.opt_price_data(opt, live, opt_strikes)

print('option strikes:', opt_strikes[0], opt_strikes[1], opt_strikes[2], \
        opt_strikes[3], '-buy call, buy put, sell call, sell put')
print('option prices:', opt_prices[0], opt_prices[1], opt_prices[2], opt_prices[3])
#print('options prices: ', buy_call_price, buy_put_price, sell_call_price, sell_put_price)


# Create Table of Hypothetical Prices with probabilities

##### right now STD range is based on current_price, you can add price pred here

df = ou.options_outcomes(perc_range, increments, \
                         current_price, current_price # future pred price here
                         , mthly_ret_std, opt_strikes, opt_prices)

df

option strikes: 180 170 190 160 -buy call, buy put, sell call, sell put
option prices: 0.0 0.0 6.3 0.0


,prices,price_perc,z-score,prob,prob_pct,buy_call_180,buy_put_170,sell_call_190,sell_put_160,profit_loss,exp_profit_loss
0,102.46,-0.40,1.580466,0.1140,0.005974,-0.0,6754.0,630.0,-5754.0,1630.0,9.737973
1,105.87,-0.38,1.501558,0.1332,0.006980,-0.0,6413.0,630.0,-5413.0,1630.0,11.378053
2,109.29,-0.36,1.422419,0.1550,0.008123,-0.0,6071.0,630.0,-5071.0,1630.0,13.240226
3,112.70,-0.34,1.343511,0.1792,0.009391,-0.0,5730.0,630.0,-4730.0,1630.0,15.307410
4,116.12,-0.32,1.264372,0.2060,0.010796,-0.0,5388.0,630.0,-4388.0,1630.0,17.596688
5,119.53,-0.30,1.185465,0.2358,0.012357,-0.0,5047.0,630.0,-4047.0,1630.0,20.142228
6,122.95,-0.28,1.106326,0.2686,0.014076,-0.0,4705.0,630.0,-3705.0,1630.0,22.944031
7,126.36,-0.26,1.027418,0.3042,0.015942,-0.0,4364.0,630.0,-3364.0,1630.0,25.985012
8,129.78,-0.24,0.948279,0.3430,0.017975,-0.0,4022.0,630.0,-3022.0,1630.0,29.299340
9,133.19,-0.22,0.869372,0.3846,0.020155,-0.0,3681.0,630.0,-2681.0,1630.0,32.852846


## Risk/Return Analysis

In [39]:
# Perform Risk Calculations

std_for_var = .5

option_combo = ou.risk_analysis(df, opt_strikes, opt_prices, std_for_var, exp_date, stock.ticker)

option_combo


,stock,exp_date,buy_call,buy_put,sell_call,sell_put,cost,exp_profit,risk_spread,bull_bear,var_std_0.5,max_loss,max_profit,risk_ratio_95th,odds_profit,odds_loss,median_profit,median_loss,buy_call_price,buy_put_price,sell_call_price,sell_put_price
0,W,2020-07-09,180 | 0.0,170 | 0.0,190 | 6.3,160 | 0.0,630.0,33.13,0,0.0,1630.0,630.0,1630.0,0.0,1.0,0.0,1630.0,NaN,0.0,0.0,6.3,0.0


## Optimizer

In [51]:
# Gather Stock Data 
stock_list = ['W', 'AMD', 'HAL', 'BAC', 'AMC', 'CZR', 'GE', 'TWTR', 'GME', 'SLB', \
              'JBLU','MGM','BBBY', 'UBER', 'LYFT', 'M', 'ACB', 'BABA', 'SQ', 'WORK', \
              'DAL', 'CCL', 'WFC', 'HTZ', 'BA', 'AAL', 'CAR']

stock_dict_final = ou.get_stock_data(stock_list)    # only run this once per day

In [52]:
# Data Check

#stock_dict_final['AMD']['ticker']
#stock_dict_final['CZR']['options']
#stock_dict_final['AMD']['hist_data'].head()
stock_dict_final['W']['option_chain']['2020-07-16']['puts'].head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,W200717P00075000,2020-06-03 19:48:08,75.0,0.23,0.00,0.00,0.00,0.000000,141,140,0.500005,False,REGULAR,USD
1,W200717P00080000,2020-06-05 17:24:23,80.0,0.33,0.25,1.15,-0.07,-17.499998,3,67,1.259769,False,REGULAR,USD
2,W200717P00085000,2020-06-03 16:57:16,85.0,0.50,0.30,0.65,0.00,0.000000,4,41,1.092778,False,REGULAR,USD
3,W200717P00090000,2020-06-05 14:37:17,90.0,0.65,0.35,0.90,0.19,41.304340,8,95,1.062505,False,REGULAR,USD
4,W200717P00095000,2020-06-05 16:30:30,95.0,0.45,0.50,0.85,-0.55,-55.000000,10,33,0.997559,False,REGULAR,USD


In [42]:
reload(ou)

<module 'option_util_V2' from 'G:\\Python_Projects\\01 StockMarketProjects\\option_util_V2.py'>

In [43]:
# OPTIMIZER V2 - INPUTS FUNCTION

def opt_inputs(opt_ticker, max_permutations, strategy_list, live): 

    optimizer_stock = stock_dict_final[opt_ticker] 

    inputs = {
     'max_permutations': max_permutations  # 0 means no cap. This is Max Permutations per STRATEGY
    ,'exp_datelist': ou.date_filter(optimizer_stock['options'],4,26) # min wks out, max wks out 
    ,'strategy_list': strategy_list #,'bear_put','bull_call']  # ['bear_put', 'bull_call', 'strangle', 'ric'] (reverse IronCondor). # list for multiple 

    ,'live': live 
    ,'past_time': 20           # time period to base STD, in business days
    ,'perc_range': .7      # for simulating hypothetical outcomes
    ,'increments': .01
    ,'std_for_var': .5     # will calculate average profit/loss at specified point std
    ,'volume': np.where(live==1,0,15)                # min volume       (15)
    ,'openInterest': np.where(live==1,0,50)          # min open interest (50)
    ,'stock': optimizer_stock
    ,'option_chain': optimizer_stock['option_chain']
    ,'ticker': optimizer_stock['ticker']
    ,'hist_data': optimizer_stock['hist_data']
    ,'current_price': optimizer_stock['hist_data']['Close'].tail(1)[0]
    }
    return inputs 

In [44]:
# MULTI PROCESSING FUNCTION

def multi_proc(inputs, current_time):
    #current_time = datetime.now().strftime('%Y-%m-%d_%H-%M')

    ticker = inputs['ticker']

    # Generate Permutations & STD data
    combos, std_data, exp_dict = ou.ric_permutations3(inputs)  # V3 is multiple strategies   # V2 is RIC only
    combo_list = np.array(combos.values.tolist())

    # Multiprocessing -- Creating smaller blocks, feed in and loop through multi processing
    stock_output = pd.DataFrame()
    if __name__ == '__main__':
        block_size = 50000  # 50000

        for i in range(math.ceil(len(combo_list)/block_size)):
            clear_output()
            combo_block = combo_list[i*block_size:(i*block_size+block_size)]
            final_values = []
            print('\nStock: ', ticker, '      Permutations: ', len(combo_list), '       Start Time: ', current_time)
            print('\nNumber of Expiry Dates: ', len(exp_dict),'\n',exp_dict)
            print('\nBlock:  ' + str(i+1)+' of  '+str(math.ceil(len(combo_list)/block_size)) + '    Block Start: '+\
                   str(i*block_size), end=' ')
            with Pool(7) as p:
                final_values = list(tqdm(p.imap(ou.stock_optimizer1, [ [combo_block[x], std_data, inputs] \
                            for x in range(len(combo_block)) ] ) , total=len(combo_block)))
            p.close()
            p.terminate()
        # list comprehension optimizer
            #final_array = np.array([ ou.stock_optimizer1( [combo_list[i], std_data, inputs] ) for i in range(len(combo_list)) ])

    #Processing Results
            block_output = pd.DataFrame( (i[0][0] for i in final_values) , columns = final_values[0][1])
            stock_output = stock_output.append(block_output, ignore_index=True)

    # Adding weighted score
    stock_output = ou.ric_score(stock_output)

    stock_output.head(50000).to_csv(rf'Output\{ticker}_{current_time}.csv')
    return stock_output

In [19]:
# Run the Optimizer!

#stonks: ['W','AMD','HAL', 'BAC','AMC', 'CZR', 'GE', 'TWTR', 'GME', 'SLB', \
#              'JBLU','MGM','BBBY', 'UBER', 'LYFT', 'M', 'ACB', 'BABA', 'SQ', 'WORK', \
#              'DAL', 'CCL', 'WFC', 'HTZ', 'BA', 'AAL', 'CAR']
# Inputs
max_permutations = 100 # per strategy, up to 4 strategies per stock. e.g. 10 stocks

opt_stock_list = ['W','AMD','HAL'] #, 'BAC', 'AMC', 'CZR', 'GE', 'TWTR', 'GME', 'SLB', \
              #'JBLU', 'MGM', 'BBBY', 'UBER', 'LYFT', 'M', 'ACB', 'BABA', 'SQ', 'WORK', \
              #'DAL', 'CCL', 'WFC', 'HTZ', 'BA', 'AAL', 'CAR']
strategy_list = ['ric', 'bear_put','bull_call','strangle']   # ric, bear_put, bull_call, strangle
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M')
live = input('\nWhat Price Do You Want? 0 Worst Bid/Ask; 1 LastTradedPrice; 2 Mid Bid/Ask:  ')
final_output = pd.DataFrame()

for i in range(len(opt_stock_list)):
    inputs = opt_inputs(opt_stock_list[i],max_permutations, strategy_list, live)
    final_output = final_output.append(multi_proc(inputs,current_time))
final_output = final_output.sort_values('score', ascending=False).reset_index(drop=True)

final_output = ou.ric_score(final_output)
final_output.to_csv(rf'Output\final_output_{current_time}.csv')

final_output.head(5)


Stock:  HAL       Permutations:  360        Start Time:  2020-06-06_15-05

Number of Expiry Dates:  4 
 {'2020-07-16': 203.0, '2020-08-20': 34.0, '2020-10-15': 122.0, '2020-07-09': 1.0}

Block:  1 of  1    Block Start: 0 

,stock,exp_date,strategy,price,buy_call,buy_put,sell_call,sell_put,cost,exp_profit,risk_spread,bull_bear,var_std_0.5,max_loss,max_profit,risk_ratio_95th,odds_profit,odds_loss,median_profit,median_loss,buy_call_vol,buy_put_vol,sell_call_vol,sell_put_vol,score
0,HAL,2020-07-16,ric,14.52,17.0 | 0.41,15.0 | 1.48,21.0 | 0.07,14.0 | 0.9,-92.0,0.03,0.25,-0.27,-42.0,-92.0,308.0,3.0,0.60,0.38,8.0,-92.0,853.0 | 1304,422.0 | 105,403.0 | 350,368.0 | 250,0.957218
1,HAL,2020-07-16,strangle,14.52,15.0 | 1.0,14.0 | 0.97,None | 0.0,None | 0.0,-197.0,0.25,0.33,0.01,-15.0,-197.0,771.0,3.0,0.46,0.54,390.0,-123.5,3716.0 | 5650,368.0 | 250,0.0 | 0,0.0 | 0,0.932950
2,HAL,2020-07-16,strangle,14.52,14.0 | 1.5,13.0 | 0.61,None | 0.0,None | 0.0,-211.0,0.24,0.35,0.17,-29.0,-211.0,857.0,3.0,0.46,0.54,378.0,-130.5,6541.0 | 6679,279.0 | 964,0.0 | 0,0.0 | 0,0.928364
3,HAL,2020-07-16,strangle,14.52,16.0 | 0.65,15.0 | 1.48,None | 0.0,None | 0.0,-213.0,0.22,0.35,-0.15,-31.0,-213.0,851.0,3.0,0.46,0.54,380.0,-132.5,739.0 | 835,422.0 | 105,0.0 | 0,0.0 | 0,0.927439
4,HAL,2020-07-16,strangle,14.52,15.0 | 1.0,13.0 | 0.61,None | 0.0,None | 0.0,-161.0,0.24,0.35,0.08,-29.0,-161.0,807.0,3.0,0.45,0.55,384.0,-130.0,3716.0 | 5650,279.0 | 964,0.0 | 0,0.0 | 0,0.927082


### Optimizer - Things To Do

 - Build algo to buy options. give it data night before. ThinkorSwim
 - Why is some cost positive?
 - Why is max loss different from cost?


### Manual Deep Dive

In [46]:
final_output = pd.read_csv(r"Output\final_output_2020-06-05_18-07.csv")
final_output.head(6)

,Unnamed: 0,stock,exp_date,strategy,price,buy_call,buy_put,sell_call,sell_put,cost,exp_profit,risk_spread,bull_bear,var_std_0.5,max_loss,max_profit,risk_ratio_95th,odds_profit,odds_loss,median_profit,median_loss,buy_call_vol,buy_put_vol,sell_call_vol,sell_put_vol,score
0,0,BABA,2020-07-16,ric,219.55,235.0 | 2.97,230.0 | 14.85,295.0 | 0.06,220.0 | 8.65,-911.0,0.49,0.10,-0.34,-411.0,-911.0,5089.0,3.0,0.66,0.34,89.0,-525.0,147.0 | 9286,54.0 | 307,18.0 | 167,247.0 | 726,0.880895
1,1,BABA,2020-07-16,ric,219.55,235.0 | 2.97,230.0 | 14.85,300.0 | 0.03,220.0 | 8.65,-914.0,0.48,0.10,-0.34,-414.0,-914.0,5586.0,3.0,0.66,0.34,86.0,-528.0,147.0 | 9286,54.0 | 307,264.0 | 2163,247.0 | 726,0.880506
2,2,BABA,2020-07-16,ric,219.55,235.0 | 2.97,230.0 | 14.85,310.0 | 0.03,220.0 | 8.65,-914.0,0.48,0.10,-0.34,-414.0,-914.0,6586.0,3.0,0.66,0.34,86.0,-528.0,147.0 | 9286,54.0 | 307,16.0 | 2229,247.0 | 726,0.880506
3,3,BABA,2020-07-16,ric,219.55,235.0 | 2.97,230.0 | 14.85,320.0 | 0.02,220.0 | 8.65,-915.0,0.48,0.10,-0.34,-415.0,-915.0,7585.0,3.0,0.66,0.34,85.0,-529.0,147.0 | 9286,54.0 | 307,51.0 | 432,247.0 | 726,0.880506
4,4,BABA,2020-07-16,ric,219.55,230.0 | 4.4,225.0 | 11.65,295.0 | 0.06,160.0 | 0.25,-1574.0,1.18,0.15,-0.18,-287.0,-1574.0,4926.0,3.0,0.46,0.54,4926.0,-946.0,937.0 | 9507,64.0 | 5650,18.0 | 167,26.0 | 3695,0.880326
5,5,BABA,2020-07-16,ric,219.55,230.0 | 4.4,225.0 | 11.65,310.0 | 0.03,160.0 | 0.25,-1577.0,1.17,0.15,-0.18,-290.0,-1577.0,6423.0,3.0,0.46,0.54,4923.0,-949.0,937.0 | 9507,64.0 | 5650,16.0 | 2229,26.0 | 3695,0.879937


In [85]:
#final_output[ (final_output['cost'] > -200)].sort_values('score', ascending=False).head(80) 
final_output[ (final_output['strategy'] == 'strangle') & final_output['cost'] > -300].sort_values('score', ascending=False).head(80) 

#final_output[ (final_output['cost'] > -200) & (final_output['strategy'] == 'ric') \
#                ].sort_values('score', ascending=False).head(15) 

final_output[ ~final_output['stock'].isin(['BABA','BAC','BA'])].head()

,Unnamed: 0,stock,exp_date,strategy,price,buy_call,buy_put,sell_call,sell_put,cost,exp_profit,risk_spread,bull_bear,var_std_0.5,max_loss,max_profit,risk_ratio_95th,odds_profit,odds_loss,median_profit,median_loss,buy_call_vol,buy_put_vol,sell_call_vol,sell_put_vol,score
2116,2116,WFC,2020-07-16,ric,31.67,27.5 | 4.95,25.0 | 0.34,30.0 | 3.05,15.0 | 0.02,-222.0,0.02,0.21,0.27,-94.5,-222.0,778.0,3.0,0.73,0.27,28.0,-173.0,4734.0 | 29860,1143 | 11534,3168.0 | 59606,119 | 2762,0.853864
2342,2342,AMD,2020-07-16,ric,53.10,65.0 | 0.55,62.5 | 10.3,85.0 | 0.05,57.5 | 6.05,-475.0,-0.19,0.22,-0.20,25.0,-475.0,1525.0,3.0,0.81,0.19,25.0,-284.0,1365 | 12190,22 | 171,130 | 5130,63 | 2486,0.852342
2419,2419,AMD,2020-07-16,ric,53.10,65.0 | 0.55,62.5 | 10.3,90.0 | 0.04,57.5 | 6.05,-476.0,-0.20,0.22,-0.20,24.0,-476.0,2024.0,3.0,0.81,0.19,24.0,-285.0,1365 | 12190,22 | 171,175 | 6227,63 | 2486,0.851953
2472,2472,AMD,2020-07-16,ric,53.10,75.0 | 0.16,62.5 | 10.3,85.0 | 0.05,60.0 | 7.95,-246.0,-0.11,0.31,-0.10,4.0,-246.0,754.0,3.0,0.90,0.10,4.0,-246.0,592 | 7270,22 | 171,130 | 5130,33 | 1928,0.851593
2503,2503,TWTR,2020-07-16,ric,34.87,32.0 | 3.95,30.0 | 0.55,34.0 | 2.52,22.0 | 0.0,-198.0,-0.10,0.16,0.32,-98.0,-198.0,602.0,3.0,0.68,0.32,2.0,-155.0,163 | 1987,532 | 3207,351 | 1871,98 | 623,0.851343


In [54]:
# Options Info
index_num = 188

dd = final_output.iloc[[index_num]]
exp_deep_dive = dd['exp_date'].values[0] #'2020-09-17'
stock_dd = dd['stock'].values[0] #'2020-09-17'
tmp_strikes = [dd['buy_call'].values[0].split()[0], dd['buy_put'].values[0].split()[0],\
                  dd['sell_call'].values[0].split()[0], dd['sell_put'].values[0].split()[0] ]

#opt_strikes_dd = []
#for x in tmp_strikes:
#    if x == 'None':
#        strike = None
#    else:
#        strike = float(x)
#    opt_strikes_dd.append(strike)
    
opt_strike_dd = [ None if x == 'None' else float(x) for x in tmp_strikes ]
            
opt_dd = stock_dict_final[f'{stock_dd}']['option_chain'][exp_deep_dive]
opt_info = pd.concat( [opt_dd['calls'][opt_dd['calls']['strike'] == opt_strikes_dd[0]], \
                       opt_dd['puts'][opt_dd['puts']['strike'] == opt_strikes_dd[1]], \
                        opt_dd['calls'][opt_dd['calls']['strike'] == opt_strikes_dd[2]], \
                         opt_dd['puts'][opt_dd['puts']['strike'] == opt_strikes_dd[3]] ] )
opt_info

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
23,BAC200821C00032000,2020-06-05 19:46:51,32.0,0.60,0.60,0.67,0.23,62.162167,1245.0,5477,0.376471,False,REGULAR,USD
22,BAC200821P00031000,2020-06-05 19:44:26,31.0,3.75,3.70,3.80,-2.60,-40.944880,215.0,16476,0.379401,True,REGULAR,USD
32,BAC200821C00041000,2020-06-05 18:53:28,41.0,0.04,0.03,0.04,0.02,100.000000,219.0,141,0.394537,False,REGULAR,USD
20,BAC200821P00029000,2020-06-05 19:48:51,29.0,2.52,2.47,2.54,-0.83,-24.776117,509.0,470,0.397955,True,REGULAR,USD


In [170]:
# DF Table Deep Dive

increments_test = .02 #  to match out to output choose .01
current_price = stock_dict_final[f'{stock_dd}']['hist_data']['Close'].tail(1)[0]

stock_data_dd, final_ret_std_dd, mthly_ret_std_dd, price_std_pct_dd, price_std_dd = \
                ou.get_std(stock_dict_final[f'{stock_dd}']['hist_data'], inputs['past_time'], exp_deep_dive) # days for STD

opt_prices_dd = ou.opt_price_data(opt_dd, inputs['live'], opt_strikes_dd)
df_test = ou.options_outcomes(.4, increments_test, \
                        current_price, current_price, # future pred price here \
                         mthly_ret_std_dd, opt_strikes_dd, opt_prices_dd)

df_test.head(90)

# look at the top 25 winning strategies for each exp date. are they bearish or bullish? tells concensus prediction
# you can also just price options strategies. puts cost more. cause there's more volatility down.options more likely strike 

,prices,price_perc,z-score,prob,prob_pct,buy_call_235.0,buy_put_230.0,sell_call_295.0,sell_put_220.0,profit_loss,exp_profit_loss
0,131.73,-0.40,2.944444,0.0032,0.000295,-297.0,8342.0,6.0,-7962.0,89.0,0.026250
1,136.12,-0.38,2.797256,0.0052,0.000479,-297.0,7903.0,6.0,-7523.0,89.0,0.042657
2,140.51,-0.36,2.650067,0.0080,0.000737,-297.0,7464.0,6.0,-7084.0,89.0,0.065626
3,144.90,-0.34,2.502878,0.0124,0.001143,-297.0,7025.0,6.0,-6645.0,89.0,0.101720
4,149.29,-0.32,2.355689,0.0184,0.001696,-297.0,6586.0,6.0,-6206.0,89.0,0.150939
5,153.69,-0.30,2.208166,0.0272,0.002507,-297.0,6146.0,6.0,-5766.0,89.0,0.223128
6,158.08,-0.28,2.060977,0.0394,0.003632,-297.0,5707.0,6.0,-5327.0,89.0,0.323207
7,162.47,-0.26,1.913788,0.0556,0.005125,-297.0,5268.0,6.0,-4888.0,89.0,0.456099
8,166.86,-0.24,1.766599,0.0772,0.007116,-297.0,4829.0,6.0,-4449.0,89.0,0.633288
9,171.25,-0.22,1.619411,0.1054,0.009715,-297.0,4390.0,6.0,-4010.0,89.0,0.864619


In [148]:

std_for_var = .5

option_combo = ou.risk_analysis(df_test, opt_strikes_dd, opt_prices_dd, std_for_var, exp_deep_dive)

option_combo

,exp_date,buy_call,buy_put,sell_call,sell_put,expected_profit,risk_spread,bull_bear,var_std_0.5,max_loss,max_profit,risk_ratio_95th,odds_profit,odds_loss,median_profit,median_loss,buy_call_price,buy_put_price,sell_call_price,sell_put_price
0,2020-06-25,385.0 | 0.08,300.0 | 3.75,390.0 | 0.02,297.5 | 3.05,-1.32,0.28,-0.12,-76.0,-76.0,424.0,3.0,0.12,0.88,174.0,-76.0,0.08,3.75,0.02,3.05


## Next Steps


0. show options volume & open interest in output

1. Build in several options strategies
    - keep same function but pass in a variable in PERMUTATIONS, RISK ANALYSIS that tailors to that options strategy
            - how is options_outcomes affected by NULL data?
        - RIC
        - Bear Put
        - Bull Call
        - Straddle

2. Pattern matching --
    - 30 day time periods. track return % from Day 1
    - Match up returns. Least distance. day 1, day 2, lowest mse
    - Match up 3 weeks. Then predict week 4
    - Variable 1: hist date range (max 5 - 10 years)
    - Variable 2: 30 day time period (45? 60?)
    - Variable 3: Future prediction range
    - Variable 4: Num to use for avg. Do a weighted avg? 
           - try normalizing returns with a min & max approach

3. Black Scholes pricing model - https://aaronschlegel.me/black-scholes-formula-python.html
    
4. Stock price predictive model
    - full list of methods is below
5. Stock Picker
    -Stocks with highest VOL 
    -Stocks with a recent huge drawdown
    -Stocks with high options volume
    -Stocks that have broken above a trading range
6. Volatility predictor 
    - how does volatility change over time
7. Stocks Price Predictor with Best Options
    - Algo that predicts future stock price for top 50 stocks
        - Highest Options Volume
        - High volatility
    - Then provides best Options to choose given that future price prediction
8. Pick Stocks for retirement account
    - Value stocks (p/e or whatever)
    - Good growth
    - Smaller market caps 



## Building Options Model

### Picking a stock ideas
 - rising volatility improves options
     - use options table to find stock w/ largest IV
 - you'll then want to rotate through top stocks and spit out best options
 - Stocks dropping in value have higher volatility
 - Predict stock price moving average 30 days out and wrap volatility around that
 


## Predict Stock Direction using:
     
     - Find highly correlated stocks. If one most more than the other. buy the lagger & short the leader. (simons strategy)
     - RSI
     - MA
     - EMA
     - Logistic Regression
     - Linear Regression
     - SVM
     - Random Forrest
     - Gradient Boosting
     - Prophet (separate notebook)
     - LSTM https://github.com/kartik-joshi/Stock-predection  
     - Back Propogation
     - Detect technical chart patterns: https://medium.com/automation-generation/algorithmically-detecting-and-trading-technical-chart-patterns-with-python-c577b3a396ed
                        - https://pythonprogramming.net/average-outcomes-as-predictions-forex-hft/?completed=/predicting-from-patterns/
         - Culmination of all of these strategies. pick majority. or just regression to get highest R2's and weight. 


https://medium.com/datadriveninvestor/predicting-stock-prices-comparison-of-different-algorithms-ab2b8fd42514

## Predict Price Movement (volatility) -- not direction:
     - hypothesis: stocks get more volatile as they drop. predict realized volatility based on 30 day prior price movement. what % up or down works best. 
     - RSI
     - MA - separation from avg could be a predictor
     - EMA
     - Logistic Regression
     - Linear Regression
     - SVM
     - Random Forrest
     - Gradient Boosting
     - Prophet (separate notebook)


In [22]:
# Do i look at std from last value or from the MA???

# reversion to the mean may or may not be a real deal
    # you can look at predicted values based on last value vs MA
    # throw a single STD around each and calc which one is more accurate


In [23]:


# use some sort of regression to predict the stock price
# or use the regression to predict the moving average and add STD to that
# to give a range

# use that data to predict probability of option

 - Maybe limit to cases where volume > X
 - 
